# Lojistik Regresyonla Tweet Duygu Analizi


## Kütüphaneler ve Veri

In [28]:
import nltk
from os import getcwd

Veri seti olarak NLTK'in [Twitter Samples](http://www.nltk.org/howto/twitter.html) veri setini kullanıyoruz. Stopword'leri de yine NLTK üzerinden indiriyoruz.

In [29]:
nltk.download('twitter_samples')

[nltk_data] Error loading twitter_samples: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [30]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

Program üzerinde çalışırken tekrar tekrar veriyi indirmek zorunda kalmamamı için aşağıdaki kod parçasını yazıyoruz.

In [31]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

### Verinin Hazırlanması
* `twitter_samples` veri seti 5.000 olumlu 5.000 olumsuz olmak üzere 10.000 Tweet verisi içerir.

In [32]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


#### Yardımcı fonksiyonlar:
* `process_tweet()`: Metni temizler, tokenlara ayırır, stopwordleri kaldırır ve kelimeleri köklerine indirger.

In [33]:
def process_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    # Kelime eklerinin, tag isaretlerinin vs. temizlenmesi 
    tweet = re.sub(r'\$\w*', '', tweet) 
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    
    # Metnin tokenlara(kelimelere) ayrilmasi
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True) 
    tweet_tokens = tokenizer.tokenize(tweet)
    
    # On islemesi gerceklesmis metnin saklanacagi dizi
    tweets_clean = []
    
    for word in tweet_tokens:
        # Stopwordlerin kaldirilmasi
        if (word not in stopwords_english and  
                word not in string.punctuation):  
            
            # Kelimenin kokune indirgenmesi
            stem_word = stemmer.stem(word) 
            tweets_clean.append(stem_word)

    return tweets_clean


* `build_freqs()`: Bu fonksiyon kelimelerin olumlu ve olumsuz cümlelerde kaçar defa geçtiğini sayarak olumlu ve olumsuz cümle frekanslarını belirlememizi sağlar.

In [34]:
def build_freqs(tweets, ys):
    
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

* Olumlu ve olumsuz tweetlerin 1000'er tanesini testte 4000'er tanesini eğitim sırasında kullanmak için ayırıyoruz.

In [35]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

* Olumlu tweetleri 1, olumsuz tweetleri 0 olarak işaretliyoruz.

In [36]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

* Frekansları bir matriste tutalım.

In [37]:
freqs = build_freqs(train_x, train_y)
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


### Tweet işleme
Tweetlerin ilk halini ve ön işleme uygulandıktan sonra algoritmaya sokacağımız halini inceleyelim.

In [38]:
print('Pozitif bir tweet: \n', train_x[0])
print('\nAynı tweetin işlenmiş hali: \n', process_tweet(train_x[0]))

Pozitif bir tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Aynı tweetin işlenmiş hali: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


# Lojistik Regresyon


### Sigmoid Fonksiyonu
Sigmoid fonksiyonu ikili sınıflandırmalarda kullanılır.
* Matematiksel olarak sigmoid fonksiyonu:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Bu fonksiyon almış olduğu inputa(z) 0 ila 1 arasında bir değer atar. 0.5'in altındaki değerler bir sınıfa 0.5'in üstündeki değerler diğer sınıfa dahil olur.


<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1200px-Logistic-curve.svg.png' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /></div>

Sigmoid fonksiyonunun kodunu yazalım.

In [39]:
def sigmoid(z):     
    h = 1/(1+np.exp(-z))
    return h

Sigmoid fonksiyonundan beklediğimiz sonuçları alabiliyor muyuz diye test edelim.

In [40]:
if (sigmoid(0) == 0.5):
    print('Doğru sonuç!')
else:
    print('Hata!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('Harika!')
else:
    print('Hata!')

Doğru sonuç!
Harika!


### Lojistik Regresyon ve Sigmoid Fonksiyonu

Lojistik regresyon lineer regresyonun çıktısına sigmoid fonksiyonu uygular ve böylece ikili sınıflandırma yapma imkanı verir.

Lineer Regresyon:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$

Lojistik Regresyon
$$ h(z) = \frac{1}{1+\exp^{-z}}$$

$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$


<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://miro.medium.com/v2/resize:fit:2000/1*zFM1ajUSh2r_sG8dQGkkeA.gif' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /></div>


### Maliyet Fonksiyonu ve Gradyan Hesabı

Lojistik regresyonda maliyet hesabı için log loss fonksiyonunu kullanıyoruz:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$  eğitim verisi sayısı
* $y^{(i)}$  i. eğitim verisinin gerçek değeri
* $h(z(\theta)^{(i)})$ modelin i. eğitim verisi için tahmini 

Tek bir eğitim için maliyet fonksiyonu:
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Model 1 tahmin ettiğinde ($h(z(\theta)) = 1$) ve $y$'de 1 olduğunda, maliyet fonksiyonu 0 üretecek yani hiç kayıp olmayacaktır. 
* Benzer şekilde, model 0 tahmin edip ($h(z(\theta)) = 0$) gerçek değer de 0 olduğu durumlarda da, maliyet fonksiyonu 0 üretecektir. 
* Ancak, model 1'e yakın bir değer tahmin edip ($h(z(\theta)) = 0.9999$) gerçek değer 0 olduğu durumlarda, maliyet fonksiyonu büyük bir değer alacaktır $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$. 
* Tahmin değeri 0'a yaklaştıkça maliyet fonksiyonunun değeri de küçülecektir.

In [41]:
# dogru sonuc aldigimizi teyit edelim
-1 * (1 - 0) * np.log(1 - 0.9999) 

9.210340371976294

In [42]:
-1 * np.log(0.0001) 

9.210340371976182

#### Ağırlıkları güncelleyelim

Ağırlık vektörünü($\theta$) güncellemek için, gradient descent(dereceli azalma) algoritmasını iteratif olarak uygulayıp parametrelerin optimum seviyeye yaklaşmasını sağlayalım.
Maliyet fonksiyonunun($J$) ağırlıklardan birine göre ($\theta_j$) türevi:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' parametresi 'm' adet eğitimden kaçıncı eğitimde olduğumuz bilgisini tutuyor 

* 'j' ağırlıkların($\theta_j$) indeksini tutuyor, $x_j$ ise $\theta_j$ ağırlığının feature bilgisini bulunduruyor.

* $\theta_j$'yı güncellemek için $\alpha$'ya göre türevinin bir kısmını çıkarıyoruz:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$

* Öğrenme derecesi($\alpha$), her güncellemenin ne kadar büyük etki edeceğini belirleyen bir parametredir.

## Gradient descent fonksiyonunun uygulanması
* `num_iters`: iterasyon sayısı 
* Her iterasyonda tek bir ağırlığı($\theta_i$) güncellemek yerine aynı anda tüm ağırlıkları bir vektör olarak güncelleyebiliriz:

$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$

* $\mathbf{\theta}$, (n+1, 1) boyutlarında bir vektördür. 'n' feature sayısıdır ve her bir terim için biaslama yapmak için fazladan bir eleman daha($\theta_0$) kullanırız (bias elemanının feature değeri($\mathbf{x_0}$)  1'dir).
* 'z', feature matrisi 'x' ile ağırlık vektörü 'theta'nın çarpımından hesaplanır.  $z = \mathbf{x}.\mathbf{\theta}$
    * $\mathbf{x}$'in boyutları: (m, n+1) 
    * $\mathbf{\theta}$'nın boyutları: (n+1, 1)
    * $\mathbf{z}$'nin boyutları: (m, 1)
* Tahmin(h), her elemana sigmoid fonksiyonu uygulanarak elde edilmiştir: 
$h(z) = sigmoid(z)$. 'h', (m,1) boyutlarındadır.
* Maliyet fonksiyonu($J$), 'y' ve 'log(h)' vektörlerinin nokta çarpımı alınarak hesaplanır. Hem 'y' hem de 'h' sütun vektörleri (m,1) olduğundan, vektörü sola aktarın, böylece bir satır vektörünün sütun vektörüyle matris çarpımı nokta çarpımını gerçekleştirir.

$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$

* Tetanın güncellenmesi de vektörleştirilmiştir. $\mathbf{x}$'in boyutları (m, n+1) olduğundan ve hem $\mathbf{h}$ hem de $\mathbf{y}$ (m, 1) boyutlarında olduğundan, matris çarpımını gerçekleştirmek için $\mathbf{x}$'in transpozesini alıp sola yerleştirmemiz gerekir, bu da ihtiyacımız olan (n+1, 1) cevabını verir:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [43]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: feature matrisi, (m,n+1) boyutlarinda
        y: inputtaki x'e karsilik gelen etiketler, (m,1) boyutlarinda
        theta: agirlik vektoru, (n+1,1) boyutlarinda
        alpha: ogrenme orani
        num_iters: modeli kac defa egitmek istiyorsak degeri bu parametreye veririz 
    Output:
        J: maliyet
        theta: son agirlik vektoru
    '''

    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # x ve theta'nin nokta carpimi
        z = np.dot(x,theta)
        
        # z'nin sigmoidi
        h = sigmoid(z)
        
        # maliyet fonksiyonunu hesaplayalim
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))

        # agirliklari guncelleyelim
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    J = float(J)
    return J, theta

## Özellik çıkarımı
* Verilen tweet listesinden iki özellik çıkarımı yapacağız: 
    * Tweetteki pozitif kelime sayısı
    * Tweetteki negatif kelime sayısı
* Daha sonra bu iki özellikle lojistik regresyonumuzu güncelleyeceğiz

### extract_features fonksiyonu
* Bu fonksiyon tek bir tweeti işler
* İlk olarak `process_tweet()` fonksiyonuyla tweeti temizleriz
* Temizlenen tweetteki her kelime için döngüye gireriz 
    * Her kelime için 'freqs' kütüphanesinde pozitif cümlelerde ve negatif cümlelerde kaçar defa geçtiği bilgisini saklarız.

In [44]:
def extract_features(tweet, freqs):

    word_l = process_tweet(tweet)
    
    # pozitif frekans, negatif frekans ve bias olmak uzere her kelimenin uc degeri mevcut(1x3)
    x = np.zeros((1, 3)) 
    
    #bias terimini 1 olarak ayarliyoruz
    x[0,0] = 1 
    
    #dizideki tum kelimeler icin donguye giriyoruz
    for word in word_l:
        
        # etiket pozitifse kelimenin pozitif frekans degerini bir arttiriyoruz
        x[0,1] += freqs.get((word, 1.0),0)
        
        # etiket negatifse kelimenin negatif frekans degerini bir arttiriyoruz
        x[0,2] += freqs.get((word, 0.0),0)
        
    assert(x.shape == (1, 3))
    return x

## Modelin eğitilmesi

Modeli eğitmek için:
* Tüm eğitimler sonucu elde ettiğiniz değerleri 'X' matrisine geçirin.
* `gradientDescent` fonksiyonunu çağırın.

In [45]:
# featurelari 'X' matrisine gecirelim
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# X'e karsilik gelen etiketleri Y'ye gecirelim
Y = train_y

# Gradient descent uygulayalim
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"Egitim sonrasi maliyet {J:.8f}.")
print(f"Ortaya cikan agirlik vektoru {[round(t, 8) for t in np.squeeze(theta)]}")

Egitim sonrasi maliyet 0.24216529.
Ortaya cikan agirlik vektoru [7e-08, 0.0005239, -0.00055517]


# Lojistik regresyonu test edelim

Modelin eğitildiği veri setinde olmayan bir tweeti lojistik regresyona sokalım.

#### `predict_tweet` fonksiyonu
Tweetin olumlu mu olumsuz mu olduğunu tahmin ettirelim.

* Tweeti işleyip kelimelerinin frekans değerlerini saptayalım.
* Eğitimi gerçekleşmiş modele tweeti sokalım ve değerini tahmin edelim

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [46]:
def predict_tweet(tweet, freqs, theta):
    
    # tweetin özellik çıkarımını yapıp x'e aktaralım
    x = extract_features(tweet,freqs)
    
    # x ve theta ile tahmini yapalım
    y_pred =sigmoid(np.dot(x,theta))
    
    return y_pred

In [47]:
# predict_tweet'i test edelim
# Olumlu tweetlerin 0.5-1 araliginda, olumsuz tweetlerin 0-0.5 araliginda olmasini bekliyoruz

tweet1 = 'I am happy'
tweet2 = 'I am bad'
print( '%s -> %f' % (tweet1, predict_tweet(tweet1, freqs, theta)))
print( '%s -> %f' % (tweet1, predict_tweet(tweet2, freqs, theta)))

I am happy -> 0.518580
I am happy -> 0.494339


## Test verileriyle performansı kontrol edelim

#### `test_logistic_regression` fonksiyonu
* Test verileri ve eğitilen modelinizin ağırlıkları göz önüne alındığında, lojistik regresyon modelinizin doğruluğunu hesaplayın.
* Test kümesindeki her tweetin değeri `predict_tweet()` fonksiyonuyla tahmin edilir
* Tahmin > 0,5 ise modelin sınıflandırmasını 1 olarak, aksi takdirde 0 olarak ayarlayalım.
* Bir tahmin, tweetingerçek değerine (test_y) eşit olduğunda doğrudur. Eşit oldukları tüm durumları toplayıp ve 'm'ye bölelim.

In [48]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: tweet listesi
        test_y: tweetin gercek degeri, boyutlar(m, 1)
        theta: agirlik vektoru, boyutlar(3, 1)
    Output: 
        accuracy: dogru siniflandirilan tweet sayisi / tum tweet sayisi
    """
    
    # tahminleri saklayacagimiz liste
    y_hat = []
    
    for tweet in test_x:
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)

    return accuracy

In [49]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Modelin dogrulugu = {tmp_accuracy:.4f}")

Modelin dogrulugu = 0.9950


# Hata Analizi
Yanlış sınıflandırdığımız tweetleri inceleyelim.

In [50]:
print('Yanlis Siniflandirilan Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('Tweet:', x)
        print('Islenmis hali:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Yanlis Siniflandirilan Tweet
Tweet: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
Islenmis hali: ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']
1	0.49996890	b'truli later move know queen bee upward bound movingonup'
Tweet: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
Islenmis hali: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48622857	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
Tweet: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
Islenmis hali: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48370665	b"i'm play brain dot braindot"
Tweet: I'm playing Brain Dots : ) #BrainDots http://t.co/aOK

# Kendi tweetlerinizi deneyin

In [51]:
your_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
y_hat = predict_tweet(your_tweet, freqs, theta)
if y_hat > 0.5:
    print('Pozitif cumle')
else: 
    print('Negatif cumle')

Negatif cumle
